In [26]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
import pdf_functions as pf

In [28]:
scanned_text = pf.read_pdf('pdf_examples/amazon_111215.pdf')

# add the contents for each page into a list
contents = pf.text_to_list(scanned_text)

# iterate over each page
by_line = []
for page in range(0, len(scanned_text.pages)):  # create a flat list of strings
    by_line += contents[page].split('\n')       # remove the line breaks

# create a dictionary of search terms
table_1_search, table_1_type = pf.table_1_search_dict()
table_2_search, table_2_type = pf.table_2_search_dict()

# create 
table_1 = pf.one_table(table_1_search, by_line)
table_2 = pf.many_table(table_2_search, by_line) 

table_1_df = pf.one_table_df(table_1)
table_2_df = pf.many_table_df(table_2)  # verifies that the length of each value list is the same in a many table

# add a column of invoice numbers for each table_2 entry
#table_2_df['invoice_number'] = table_1['invoice_number'][0]
table_2_df.insert(0, 'invoice_number', table_1['invoice_number'][0])

# format dates to standard YYYY-MM-DD
formatted_date_str = pf.format_date(table_1['invoice_date'][0], '%B %d, %Y')  # November 11, 2015
name_base = table_1['company_name'][0].replace('.', '_') + '_' + formatted_date_str

pf.save_as_csv(table_1, table_1_df, name_base, 'overview')
pf.save_as_csv(table_1, table_2_df, name_base, 'products')


The file has 2 pages.
